In [42]:
import os
import logging

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Log setting
logging.basicConfig(format="%(asctime)s %(levelname)s %(message)s", datefmt="%H:%M:%S", level=logging.INFO)

# Change display.max_rows to show all features.
pd.set_option("display.max_rows", 85)

In [51]:
DIR_PATH = "datasets"
FILE_PATH = os.path.join(DIR_PATH, "MachineLearningCVE.csv")
VALIDATION_SET_FILE_PATH = os.path.join(DIR_PATH, "Friday-WorkingHours-Afternoon-PortScan-validation.pcap_ISCX.csv")

In [55]:
def _label_encoding() -> LabelEncoder:
    # Create Label Encoder
    le = LabelEncoder()

    # Read Label column from all dataset files
    labels = pd.read_csv(FILE_PATH, usecols=['Label'], skipinitialspace=True)

    # Fit the labels data to Label Encoder
    le.fit(labels.Label)

    # Saving the label encoder
    np.save(os.path.join(DIR_PATH, 'label_encoder.npy'), le.classes_)

    # Log the result.
    logging.info("Total rows: {}".format(labels.shape))
    logging.info("Class distribution:\n{}\n".format(labels.Label.value_counts()))

    return le


def _process(df: pd.DataFrame, le: LabelEncoder):
    # Label encoding
    df.Label = le.transform(df.Label)

    # Fill NaN with average value of each class in this dataset
    nan_rows = df[df.isna().any(axis=1)].shape[0]
    logging.info("Fill NaN in {} rows with average value of each class.".format(nan_rows))
    df.iloc[:, df.columns != "Label"] = df.groupby("Label").transform(lambda x: x.fillna(x.mean()))

    # Change inf value with maximum value of each class
    inf_rows = df[df.isin([np.inf]).any(axis=1)].shape[0]
    logging.info("Replace Inf in {} rows with maximum value of each class.".format(inf_rows))
    # Temporary replace inf with NaN
    df = df.replace([np.inf], np.nan)
    # Replace inf with maximum value of each class in this dataset
    df.iloc[:, df.columns != "Label"] = df.groupby("Label").transform(lambda x: x.fillna(x.max()))

    # Change negative value with minimum positive value of each class
    logging.info("Replace negative values with minimum value of each class.")
    # Temporary replace negative value with NaN
    df[df < 0] = np.nan
    # Replace negative value with minimum value of each class in this dataset
    df.iloc[:, df.columns != "Label"] = df.groupby("Label").transform(lambda x: x.fillna(x.min()))

    return df

def _feature_selection(df: pd.DataFrame):
    X = df.drop(columns=['Label'])

    # Create correlation matrix
    corr_matrix = X.corr().abs()
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    # Find features with correlation greater than 0.9
    to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]

    logging.info(to_drop)

    df.drop(to_drop, axis=1, inplace=True)
    return df


def _split_train_test(df: pd.DataFrame):
    # Sampling the dataset
    x = df.iloc[:, df.columns != 'Label']
    y = df[['Label']]

    x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.20,
                                                        random_state=np.random.randint(10))

    del x, y

    train = pd.concat([x_train, y_train], axis=1, sort=False)
    test = pd.concat([x_test, y_test], axis=1, sort=False)

    return train, test


def _to_csv(df: pd.DataFrame, saving_path: str):
    # if file does not exist write header
    if not os.path.isfile(saving_path):
        df.to_csv(saving_path, index=False)
    # else it exists so append without writing the header
    else:
        df.to_csv(saving_path, index=False, mode='a', header=False)


def _preprocessing_all(le: LabelEncoder, chunksize=1000000):
    # Preprocess all file
    for chunk in pd.read_csv(FILE_PATH, skipinitialspace=True, chunksize=chunksize):
        # train, test = _split_train_test(_feature_selection(_process(chunk, le)))
        train, test = _split_train_test(_process(chunk, le))
        _to_csv(train, os.path.join(DIR_PATH, "train_MachineLearningCVE.csv"))
        _to_csv(test, os.path.join(DIR_PATH, "test_MachineLearningCVE.csv"))

def _preprocessing_validation(le: LabelEncoder, chunksize=1000000):
    # Preprocess all file
    for chunk in pd.read_csv(VALIDATION_SET_FILE_PATH, skipinitialspace=True, chunksize=chunksize):
        validation = _process(chunk, le)
        _to_csv(validation, os.path.join(DIR_PATH, "validation_MachineLearningCVE.csv"))

In [54]:
label_encoder = _label_encoding()

18:56:27 INFO Total rows: (2748030, 1)
18:56:28 INFO Class distribution:
BENIGN                      2214224
DoS Hulk                     231073
PortScan                     135090
DDoS                         128027
DoS GoldenEye                 10293
FTP-Patator                    7938
SSH-Patator                    5897
DoS slowloris                  5796
DoS Slowhttptest               5499
Bot                            1966
Web Attack-Brute Force         1507
Web Attack-XSS                  652
Infiltration                     36
Web Attack-Sql Injection         21
Heartbleed                       11
Name: Label, dtype: int64



In [46]:
_preprocessing_all(label_encoder, 2500000)

16:05:41 INFO Fill NaN in 1344 rows with average value of each class.
16:06:44 INFO Replace Inf in 2718 rows with maximum value of each class.
16:07:28 INFO Replace negative values with minimum value of each class.
16:09:35 INFO Fill NaN in 6 rows with average value of each class.
16:09:38 INFO Replace Inf in 60 rows with maximum value of each class.
16:09:41 INFO Replace negative values with minimum value of each class.


In [56]:
_preprocessing_validation(label_encoder, 2500000)

18:58:26 INFO Fill NaN in 3 rows with average value of each class.
18:58:26 INFO Replace Inf in 57 rows with maximum value of each class.
18:58:27 INFO Replace negative values with minimum value of each class.


In [47]:
logging.info("*** END ***")

16:09:53 INFO *** END ***
